In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
import plotly.express as px

In [2]:
data = pd.read_csv("learn.csv")

In [3]:
del data["Unnamed: 0"]

In [4]:
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
data.birth_year = pd.to_datetime(data.birth_year)
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [15]:
x = data.copy()

In [16]:
c1 = pd.get_dummies(x.income_type)

In [17]:
x = pd.concat([x, c1], axis = 1)

In [18]:
del x['income_type']

In [19]:
x = x[["application_id", "loan_limit", "loan_rate", "credit_score", "company_enter_month","insert_time",
                  "desired_amount", "EARNEDINCOME", "is_applied"]]

In [22]:
test_data=x[x['insert_time']>='2022-05-16']
train_data = x[x['insert_time']<'2022-05-16']

In [23]:
test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_26970/4276231571.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [24]:
train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_26970/1691262433.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [25]:
test_label = test_data["is_applied"]
del test_data["is_applied"]

In [26]:
train_label = train_data["is_applied"]
del train_data["is_applied"]

In [30]:
test_data

,loan_limit,loan_rate,credit_score,company_enter_month,desired_amount,EARNEDINCOME
0,8000000.0,15.8,550.0,2016-07-01,28000000.0,1
1,1000000.0,10.3,660.0,2022-04-01,20000000.0,1
2,100000000.0,17.9,1000.0,2019-03-01,500000000.0,0
3,50000000.0,16.4,720.0,2016-03-01,32000000.0,1
4,40000000.0,19.9,640.0,2021-11-01,42000000.0,1
...,...,...,...,...,...,...
134774,15000000.0,15.7,650.0,2010-02-01,8000000.0,1
134775,33000000.0,19.7,610.0,2022-02-01,20000000.0,1
134776,25000000.0,19.1,650.0,2021-07-01,3000000.0,0
134777,26000000.0,19.9,670.0,2016-10-01,5000000.0,1


In [12]:
from catboost import CatBoostClassifier, Pool

In [32]:
xxx = x.groupby("application_id").max()

In [34]:
k = xxx.is_applied.mean()

In [13]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [35]:
train = Pool(
    data = train_data,
    label = train_label,
    weight = train_label * (1-2*k) + k
)

In [ ]:
1

In [82]:
model.fit(train)

0:	learn: 0.6657609	total: 20.8ms	remaining: 20.8ms
1:	learn: 0.6596111	total: 37.3ms	remaining: 0us


In [83]:
p = model.predict(test_data)

In [39]:
from sklearn.metrics import f1_score

In [84]:
f1_score(test_label, p)

0.6215514235495877

In [43]:
model2 = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [44]:
y = x.copy()

In [45]:
kk = y.groupby("application_id").max()

In [46]:
kkk = kk.loc[kk.is_applied == 1]

In [47]:
appyes = kkk.index

In [48]:
y = y.loc[y.application_id.isin(appyes)]

In [49]:
gggg = y.is_applied.mean()

In [50]:
y = y[["loan_limit", "loan_rate", "credit_score", "company_enter_month","insert_time",
                  "desired_amount", "EARNEDINCOME", "is_applied"]]

In [51]:
trainy = y[y['insert_time']<'2022-05-16']
testy = y[y['insert_time']>= '2022-05-16']

In [52]:
trainl = trainy["is_applied"]
testl = testy["is_applied"]
del trainy["insert_time"]

del testy["insert_time"]


In [53]:
del trainy["is_applied"]
del testy["is_applied"]

In [54]:
traintrain = Pool(
    data = trainy,
    label = trainl,
    weight = trainl * (1-2*gggg) + gggg
    )

In [55]:
model2.fit(traintrain)

0:	learn: 0.5937376	total: 196ms	remaining: 196ms
1:	learn: 0.5643237	total: 328ms	remaining: 0us


In [56]:
pp = model.predict(testy)

In [59]:
len(pp)

895693

In [57]:
f1_score(testl, pp)

0.2721819574023138

In [61]:
z = x.copy()
z["predict"] = 0
z = z.loc[z.insert_time > "2022-05-16"]

In [63]:
yes = z.loc[z.application_id.isin(appyes)]

In [65]:
yes['score'] = 0

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_26970/2846301860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yes['score'] = 0


In [66]:
yes

,application_id,loan_limit,loan_rate,credit_score,company_enter_month,insert_time,desired_amount,EARNEDINCOME,is_applied,predict,score
8395658,2136969,13000000.0,17.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,0.0,0,0
8395659,2136969,5000000.0,14.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,0.0,0,0
8395660,2136969,3000000.0,12.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,1.0,0,0
8395661,824356,10000000.0,17.5,630.0,2020-10-01,2022-05-16 00:11:51,5000000.0,1,1.0,0,0
8395662,993637,5000000.0,16.3,700.0,2022-06-01,2022-05-16 00:12:14,1000000.0,0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10264369,502797,1000000.0,9.9,620.0,2021-03-01,2022-05-31 23:53:43,28000000.0,1,0.0,0,0
10264370,502797,21000000.0,14.2,620.0,2021-03-01,2022-05-31 23:53:43,28000000.0,1,0.0,0,0
10264371,502797,13000000.0,16.8,620.0,2021-03-01,2022-05-31 23:53:43,28000000.0,1,0.0,0,0
10264372,502797,5000000.0,14.9,620.0,2021-03-01,2022-05-31 23:53:43,28000000.0,1,0.0,0,0


In [68]:
xxx = yes[["loan_limit", "loan_rate", "credit_score", 
           "company_enter_month", "desired_amount", "EARNEDINCOME"]]

In [69]:
yes["score"] = model2.predict_proba(xxx)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_26970/2452806664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yes["score"] = model2.predict_proba(xxx)


In [71]:
yesyes = yes.copy()

In [72]:
k = yesyes.sort_values(by = 'score').drop_duplicates("application_id", keep = "last").index

In [73]:
z.loc[k, "predict"] = 1

In [74]:
z

,application_id,loan_limit,loan_rate,credit_score,company_enter_month,insert_time,desired_amount,EARNEDINCOME,is_applied,predict
8395658,2136969,13000000.0,17.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,0.0,0
8395659,2136969,5000000.0,14.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,0.0,1
8395660,2136969,3000000.0,12.5,650.0,2022-06-01,2022-05-16 00:11:50,2000000.0,0,1.0,0
8395661,824356,10000000.0,17.5,630.0,2020-10-01,2022-05-16 00:11:51,5000000.0,1,1.0,1
8395662,993637,5000000.0,16.3,700.0,2022-06-01,2022-05-16 00:12:14,1000000.0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
10264381,455157,3000000.0,18.0,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0
10264382,455157,3000000.0,17.5,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0
10264383,455157,5000000.0,18.4,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0
10264384,455157,3000000.0,14.5,610.0,1997-04-01,2022-05-31 23:54:27,50000000.0,1,0.0,0


In [75]:
f1_score(z.is_applied, z.predict)

0.2667603372139703